In [18]:
import pandas as pd
from collections import defaultdict
from collections import Counter
import matplotlib.pyplot as plt
from xgboost import XGBClassifier
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedShuffleSplit
#from sklearn.model_selection import train_test_split
#from sklearn.model_selection import StratifiedKFold



from sklearn import svm, datasets
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import label_binarize
from sklearn.multiclass import OneVsRestClassifier
from scipy import interp

In [2]:
%matplotlib inline  
%autosave 300
pd.options.display.max_columns = None
feature_names =defaultdict(lambda:'None')

Autosaving every 300 seconds


# Classification based on published Data:

Risk High/low set at 38.40 Readiness high/low set at 46.51


In [3]:
def categorize(row):
    if row['overall_risk']== 1 and row['overall_readiness']==1:
        return 4
    elif row['overall_risk']== 1 and row['overall_readiness']==0:
        return 3
    elif row['overall_readiness'] == 1:
        return 2
    else:
        return 1
    
risk_ready=pd.read_csv('data/Overall Risk & Readiness Scores.csv')

risk_ready_old=risk_ready.columns
new_names=['city', 'state', 'geo_id','overall_risk', 'overall_readiness']
risk_ready.columns = new_names
    
#Puerto Rico is being dropped at rows [19, 35, 193, 220] because of data inconsistency
risk_ready.drop([19, 35, 193, 220], inplace = True)
risk_ready['overall_risk'] = np.where(risk_ready['overall_risk']>=.3840, 1, 0)
risk_ready['overall_readiness'] = np.where(risk_ready['overall_readiness']>=.4651, 1, 0)
risk_ready['risk_ready_cat']=risk_ready.apply(lambda row: categorize(row), axis=1)


risk_ready.to_csv('data/transformed_data/risk_and_readiness_target_data.csv')
risk_ready_dict = dict(zip(new_names, risk_ready_old))
feature_names['risk_ready']=risk_ready_dict



Clean main data File

In [4]:
city_ind = pd.read_csv('data/City Indicators.csv')     


city_ind_old=city_ind.columns
new_names=['city','state','geo_id','GDP_water_all','perc_fff','d_plan','w_plan','base_w_stress','invest','tax_break','corruption','perc_vote','innovate','climate_real','hs_ed','pop_dens','w_quality','debt','flood_pop','flood_build','flood_car','impervious','beds_1000','alone_65','child_5','health_ins','older_1999','mobile_home','rent_50plus','trees','heating','disabled','poverty','older_1979','work_outside','no_car','pop_under_1ft','pop_under_3ft','impact_plus1ft','impact_plus3ft', 'col_empty','lat','long','county','region','city_km2','pop_2010','pop_2011','pop_2012','pop_2013','pop_2014','pop_2015','median_income']
city_ind.columns = new_names

city_ind.drop(columns =['col_empty'], inplace= True)
indicators_dict = dict(zip(city_ind.columns, city_ind_old))



#Puerto Rico is being dropped at rows [19, 35, 193, 220] because of data inconsistency
city_ind.drop([19, 35, 193, 220], inplace = True)
#two cities had their order of magnitude incorrect for % of people in floodzone. Their neighboring values match the lower magnitude'
city_ind=city_ind.replace(412.9208681,41.29208681)
city_ind=city_ind.replace(101.6959823,10.16959823)
city_ind.replace('Bayam<U+0086>n','Bayamon')
city_ind['risk_ready_cat']= risk_ready['risk_ready_cat']

city_ind['d_plan'] = city_ind['d_plan'].map({'Yes': 1, 'No': 0})
city_ind['w_plan'] = city_ind['w_plan'].map({'Yes': 1, 'No': 0})
cities=city_ind['city']
#cities.to_csv('data/transformed_data/cities.csv')
city_ind.drop(columns=['city','state','geo_id','county', 'region','pop_2010', 'pop_2011', 'pop_2012', 'pop_2013', 'pop_2014'], inplace=True)

city_ind.fillna(0, inplace=True)


#city_ind.to_csv('data/transformed_data/city_indicators_cleaned.csv')
city_ind.groupby('risk_ready_cat').count()


,GDP_water_all,perc_fff,d_plan,w_plan,base_w_stress,invest,tax_break,corruption,perc_vote,innovate,climate_real,hs_ed,pop_dens,w_quality,debt,flood_pop,flood_build,flood_car,impervious,beds_1000,alone_65,child_5,health_ins,older_1999,mobile_home,rent_50plus,trees,heating,disabled,poverty,older_1979,work_outside,no_car,pop_under_1ft,pop_under_3ft,impact_plus1ft,impact_plus3ft,lat,long,city_km2,pop_2015,median_income
risk_ready_cat,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,48,48,48,48,48,48,48,48,48,48,48,48,48,48,48,48,48,48,48,48,48,48,48,48,48,48,48,48,48,48,48,48,48,48,48,48,48,48,48,48,48,48
2,80,80,80,80,80,80,80,80,80,80,80,80,80,80,80,80,80,80,80,80,80,80,80,80,80,80,80,80,80,80,80,80,80,80,80,80,80,80,80,80,80,80
3,87,87,87,87,87,87,87,87,87,87,87,87,87,87,87,87,87,87,87,87,87,87,87,87,87,87,87,87,87,87,87,87,87,87,87,87,87,87,87,87,87,87
4,59,59,59,59,59,59,59,59,59,59,59,59,59,59,59,59,59,59,59,59,59,59,59,59,59,59,59,59,59,59,59,59,59,59,59,59,59,59,59,59,59,59


In [6]:

trees_cat=city_ind['risk_ready_cat']
trees_data = city_ind.drop(columns=['risk_ready_cat'])
tree_features=trees_data.columns
#trees_cat.to_csv('data/transformed_data/classification_labels.csv')
#trees_data.to_csv('data/transformed_data/classification_data.csv')
#tree_features.to_csv('data/transformed_data/classification_features.csv')


type(tree_features)

pandas.core.indexes.base.Index

    prob=rf.predict_proba(X_test)
    fpr,tpr,thresholds = roc_curve(y_test, prob[:, 1])
    tprs.append(interp(mean_fpr, fpr, tpr))
    tprs[-1][0] = 0.0
    roc_auc = auc(fpr, tpr)
    aucs.append(roc_auc)
    plt.plot(fpr, tpr, lw=1, alpha=0.3,
             label='ROC fold %d (AUC = %0.2f)' % (i, roc_auc))
    i += 1

    plt.plot([0, 1], [0, 1], linestyle='--', lw=2, color='r',
         label='Chance', alpha=.8)

mean_tpr = np.mean(tprs, axis=0)
mean_tpr[-1] = 1.0
mean_auc = auc(mean_fpr, mean_tpr)
std_auc = np.std(aucs)
plt.plot(mean_fpr, mean_tpr, color='b',
         label=r'Mean ROC (AUC = %0.2f $\pm$ %0.2f)' % (mean_auc, std_auc),
         lw=2, alpha=.8)

std_tpr = np.std(tprs, axis=0)
tprs_upper = np.minimum(mean_tpr + std_tpr, 1)
tprs_lower = np.maximum(mean_tpr - std_tpr, 0)
plt.fill_between(mean_fpr, tprs_lower, tprs_upper, color='grey', alpha=.2,
                 label=r'$\pm$ 1 std. dev.')

plt.xlim([-0.05, 1.05])
plt.ylim([-0.05, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic example')
plt.legend(loc="lower right")
plt.show()


In [8]:
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.multiclass import OneVsRestClassifier
from sklearn import metrics

import numpy as np
from scipy import interp
import matplotlib.pyplot as plt
import pickle

from sklearn import svm, datasets
from sklearn.metrics import roc_curve, auc
X=np.array(trees_data)
y=np.array(trees_cat)
sss = StratifiedShuffleSplit(n_splits=5, random_state=42)
sss.get_n_splits(X,y)
split_indices=list(sss.split(X,y))
#print(split_indices)
     
sss_rf_scores =[]

'''
RandomForestClassifier(n_estimators=’warn’, criterion=’gini’, max_depth=None,
min_samples_split=2, min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_features=’auto’, max_leaf_nodes=None,
min_impurity_decrease=0.0, min_impurity_split=None, bootstrap=True, oob_score=False, 
n_jobs=None, random_state=None, verbose=0, warm_start=False, class_weight=None)
'''
n=155
c = None
m=2
f='auto'
for train_index, test_index in sss.split(X, y):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    sss_rf = OneVsRestClassifier(RandomForestClassifier( n_jobs=-1, max_features = f, 
            max_depth= c, n_estimators=n, min_samples_split=m,random_state=38))
    sss_rf.fit(X_train, y_train)
    y_hat=sss_rf.predict(X_test)
    #print(y_hat,y_test)
    score=sss_rf.score(X_test,y_test)
    sss_rf_scores.append(score)
    #print(score)
    
sss_rf_mean_accuracy=sum(sss_rf_scores)/len(sss_rf_scores)
print(f'sss_rf mean accuracy= {sss_rf_mean_accuracy}')
sss_rf_feat=pd.DataFrame()

sss_rf_feat['features']=trees_data.columns

from sklearn.metrics import confusion_matrix
print(y_test)
print(y_hat)
tn, fp, fn, tp = confusion_matrix(y_test, y_hat)
print(f'TP= {tp}, fp={fp}')
#sss_rf mean accuracy= 0.7571428571428571
filename = 'data/models/oneVRest_Random_Forest.sav'
pickle.dump(sss_rf, open(filename, 'wb'))   


sss_rf mean accuracy= 0.7428571428571428
[3 2 2 1 3 2 3 4 4 4 1 3 3 1 1 3 1 3 2 3 2 2 4 2 4 4 3 2]
[3 2 2 2 3 2 3 2 3 4 3 3 3 1 2 3 2 3 2 3 2 2 4 2 4 2 3 2]
TP= [0 2 1 3], fp=[0 8 0 0]


In [ ]:
from sklearn.model_selection import train_test_split
trees_cat=city_ind['risk_ready_cat']
trees_data = city_ind.drop(columns=['risk_ready_cat'])
X_train, X_test, y_train, y_test = train_test_split(trees_data, trees_cat, test_size=0.25, random_state=42)
rf_scores=[]

n=125
c = 8
m=10
f='auto'

rf = OneVsRestClassifier(RandomForestClassifier(max_features = f,
    max_depth= c, n_estimators=n, min_samples_split=m,
    random_state=38,n_jobs=-1))
rf.fit(X_train, y_train)
y_hat=rf.predict(X_test)
#print(y_hat,y_test)
score=rf.score(X_test,y_test)
print(score)
    

rf_feat=pd.DataFrame()

rf_feat['features']=trees_data.columns
#print (y_hat, y_test)
#rf_feat

from sklearn.model_selection import train_test_split
trees_cat=city_ind['risk_ready_cat']
trees_data = city_ind.drop(columns=['risk_ready_cat'])
X_train, X_test, y_train, y_test = train_test_split(trees_data, trees_cat, test_size=0.25, random_state=42)
rf_scores=[]

n=19
c = 10
m=13
f=8

rf = RandomForestClassifier(max_features = f,
    max_depth= c, n_estimators=n, 
                            min_samples_split=m,
    random_state=38,n_jobs=-1)
rf.fit(X_train, y_train)
y_hat=rf.predict(X_test)
#print(y_hat,y_test)
score=rf.score(X_test,y_test)
print(score)
print()
tn, fp, fn, tp = confusion_matrix(y_test, y_hat)
print(f'TP= {tp}, fp={fp}')
rf_feat=pd.DataFrame()

rf_feat['features']=trees_data.columns
#print (y_hat, y_test)
#rf_feat

from sklearn.ensemble import AdaBoostClassifier
'''
AdaBoostClassifier(base_estimator=None, n_estimators=50, learning_rate=1.0, algorithm=’SAMME.R’, 
random_state=None
'''
#n_estimators =49, learning_rate = 1.1
n=49
le=1.1
ada_scores =[]
for train_index, test_index in sss.split(X, y):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    ada = OneVsRestClassifier(AdaBoostClassifier(n_estimators =n, learning_rate = le, random_state=38))
    ada.fit(X_train, y_train)
    y_hat=ada.predict(X_test)
    score=ada.score(X_test,y_test)
    ada_scores.append(score)
    
    
ada_mean_accuracy=sum(ada_scores)/len(ada_scores)

print(f'ada boost mean accuracy= {ada_mean_accuracy}')
ada_feat=pd.DataFrame()

ada_feat['features']=trees_data.columns
#ada_feat['Importance']=rf.feature_importances_

#ada boost mean accuracy= 0.7285714285714285

                        

data = np.random.rand(5, 10)  # 5 entities, each contains 10 features
label = np.random.randint(2, size=5)  # binary target
dtrain = xgb.DMatrix(data, label=label)

from sklearn.ensemble import AdaBoostClassifier
'''
AdaBoostClassifier(base_estimator=None, n_estimators=50, learning_rate=1.0, algorithm=’SAMME.R’, 
random_state=None
'''
#n_estimators =49, learning_rate = 1.1
n=49
le=1.1
#ada_scores =[]



ada_l = AdaBoostClassifier(n_estimators =n, learning_rate = le, random_state=38)
ada_l.fit(X_train, y_train)
y_hat=ada_l.predict(X_test)
score=ada_l.score(X_test,y_test)
#ada_scores_l.append(score)

    
#ada_mean_accuracy=sum(ada_scores)/len(ada_scores)
tn, fp, fn, tp = confusion_matrix(y_test, y_hat)
print(f'TP= {tp}, fp={fp}')
print(f'ada_last accuracy= {score}')
ada_feat=pd.DataFrame()

ada_feat['features']=trees_data.columns
#ada_feat['Importance']=rf.feature_importances_

#ada boost mean accuracy= 0.7285714285714285

In [21]:
print(__doc__)

import numpy as np
import matplotlib.pyplot as plt
from itertools import cycle

from sklearn import svm, datasets
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import label_binarize
from sklearn.multiclass import OneVsRestClassifier
from scipy import interp

dataset = pd.read_csv('data/winequality.csv')
dataset.isnull().any() #does a True/False for nulls
dataset = dataset.fillna(method='ffill') 

X = dataset.drop(['quality'], axis=1).values 
y = dataset['quality'].values

# Binarize the output
y = label_binarize(y, classes=[1,2,3,4,5])
n_classes = y.shape[1]


# shuffle and split training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.5,
                                                    random_state=0)

# Learn to predict each class against the other
classifier = OneVsRestClassifier(svm.SVC(kernel='linear', probability=True,
                                 random_state=42))
y_score = classifier.fit(X_train, y_train).decision_function(X_test)

# Compute ROC curve and ROC area for each class
fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(y_test[:, i], y_score[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

# Compute micro-average ROC curve and ROC area
fpr["micro"], tpr["micro"], _ = roc_curve(y_test.ravel(), y_score.ravel())
roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])

# Plot of a ROC curve for a specific class

plt.figure()
lw = 2
plt.plot(fpr[2], tpr[2], color='darkorange',
         lw=lw, label='ROC curve (area = %0.2f)' % roc_auc[2])
plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic example')
plt.legend(loc="lower right")
plt.show()



#Plot ROC curves for the multiclass problem

# Compute macro-average ROC curve and ROC area

# First aggregate all false positive rates
all_fpr = np.unique(np.concatenate([fpr[i] for i in range(n_classes)]))

# Then interpolate all ROC curves at this points
mean_tpr = np.zeros_like(all_fpr)
for i in range(n_classes):
    mean_tpr += interp(all_fpr, fpr[i], tpr[i])

# Finally average it and compute AUC
mean_tpr /= n_classes

fpr["macro"] = all_fpr
tpr["macro"] = mean_tpr
roc_auc["macro"] = auc(fpr["macro"], tpr["macro"])

# Plot all ROC curves
plt.figure()
plt.plot(fpr["micro"], tpr["micro"],
         label='micro-average ROC curve (area = {0:0.2f})'
               ''.format(roc_auc["micro"]),
         color='deeppink', linestyle=':', linewidth=4)

plt.plot(fpr["macro"], tpr["macro"],
         label='macro-average ROC curve (area = {0:0.2f})'
               ''.format(roc_auc["macro"]),
         color='navy', linestyle=':', linewidth=4)

colors = cycle(['aqua', 'darkorange', 'cornflowerblue'])
for i, color in zip(range(n_classes), colors):
    plt.plot(fpr[i], tpr[i], color=color, lw=lw,
             label='ROC curve of class {0} (area = {1:0.2f})'
             ''.format(i, roc_auc[i]))

plt.plot([0, 1], [0, 1], 'k--', lw=lw)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Some extension of Receiver operating characteristic to multi-class')
plt.legend(loc="lower right")
plt.show()



dataset = pd.read_csv('data/winequality.csv')
dataset.isnull().any() #does a True/False for nulls
dataset = dataset.fillna(method='ffill') 

X = dataset.drop(['quality'], axis=1).values 
y = dataset['quality'].values

# Binarize the output
y = label_binarize(y, classes=[1,2,3,4,5])
n_classes = y.shape[1]


# shuffle and split training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.5,
                                                    random_state=0)

# Learn to predict each class against the other
classifier = OneVsRestClassifier(svm.SVC(kernel='linear', probability=True,
                                 random_state=42))
y_score = classifier.fit(X_train, y_train).decision_function(X_test)



FileNotFoundError: [Errno 2] File b'data/winequality.csv' does not exist: b'data/winequality.csv'